# 1. Setup Mario

In [ ]:
# Installing Dependencies
%pip install gym_super_mario_bros==7.3.2
%pip install nes_py==8.1.8

In [ ]:
# imports
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
# Import symplified controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [ ]:
# Setup environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [ ]:
# Start and stop the environment
done = True

for frame in range(100000):
    if done:
        env.reset()
    
    # Do random actions
    state, reward, done, info = env.step(env.action_space.sample())
    env.render() # Render the environment

env.close()

In [ ]:
env.close()

# 2. Preprocess Environment

In [ ]:
# Install pytorch and stable baselines3
%pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
%pip install stable-baselines3[extra]

In [ ]:
# Import Greyscale and stack frames
from gym.wrappers import GrayScaleObservation
# Import Vectorization Wrappers
from stable_baselines3.common.vec_env.vec_frame_stack import VecFrameStack
from stable_baselines3.common.vec_env.dummy_vec_env import DummyVecEnv
# from matplotlib import pyplot as plt
from matplotlib import pyplot as plt

In [ ]:
env = gym_super_mario_bros.make('SuperMarioBros-v0') # Create the environment
env = JoypadSpace(env, SIMPLE_MOVEMENT) # Use simplified controls
env = GrayScaleObservation(env, keep_dim=True) # Greyscale the env
env = DummyVecEnv([lambda: env]) # Create a dummy environment
env = VecFrameStack(env, 4, channels_order='last') # Stack 4 frames


In [ ]:
plt.imshow(state[0]) # Plot the state

# 3. Train the RL Model

In [ ]:
import os
from stable_baselines3 import PPO # Import PPO for the algorithm
from stable_baselines3.common.callbacks import BaseCallback # For Saving the model

In [ ]:
# Saving The Model
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
# Directories to save data
SAVE_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=SAVE_DIR)

In [ ]:
# The AI Model
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, n_steps=512)

In [ ]:
# Train the AI model
model.learn(total_timesteps=1000000, callback=callback)

# 4. Test it out